# Resumen

Actualmente, **sin criterios de racionalizacion** tendriamos que cerrar una brecha de **5,743 plazas** y costaria **S/ 89,832,006**. Sin embargo si se usan los siguientes **criterios de ordenamiento**:

1) Nombrados excedentes se reubican dentro de su UGEL

2) Las plazas no nombradas excedentes se reubican a nivel nacional para cubrir los requerimientos

La brecha seria de **2,976 plazas** con un costo **S/46,550,592** lo que implica un **ahorro** de **2,767 plazas** equivalentes a **S/ 43,281,414**

In [1]:
#Libraries
import os 
import pandas as pd
import numpy as np

os.chdir('D:\Brecha-no-docente') # Directorio de trabajo

In [2]:
df = pd.read_stata('Resultados\Brecha con costo.dta')

C:\Users\USUARIO\Anaconda3\lib\site-packages\pandas\io\stata.py:1457: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [3]:
identificacion = ['d_dpto', 'region', 'pliego', 'codue', 'unidadejecutora', 'codooii', 'nombentidad', 'tipo_entidad']
datos = ['pers_limp_mant_n', 'pers_limp_mant_c', 'pers_limp_mant', 'opt_pers_limp_mant', 'exd_pers_limp_mant', 'req_pers_limp_mant',
        'pers_vigilancia_n', 'pers_vigilancia_c', 'pers_vigilancia', 'opt_pers_vigilancia', 'exd_pers_vigilancia', 'req_pers_vigilancia', 
         'coord_adm_ie_n', 'coord_adm_ie_c', 'coord_adm_ie', 'opt_coord_adm_ie', 'exd_coord_adm_ie', 'req_coord_adm_ie',
         'oficinista_n', 'oficinista_c', 'oficinista', 'opt_oficinista', 'exd_oficinista', 'req_oficinista',
         'secretario_n', 'secretario_c', 'secretario', 'opt_secretario', 'exd_secretario', 'req_secretario'
        ]

In [4]:
df_ugel = df.groupby(identificacion)[datos].sum()
df_ugel.reset_index(inplace = True)

for x in datos:
    df_ugel[x] = df_ugel[x].astype(int)

In [5]:
personal = ['pers_limp_mant','pers_vigilancia', 'coord_adm_ie', 'oficinista', 'secretario']

# Desagrego el personal excedente
for x in personal:
    df_ugel[f'exd_{x}_c'] = df_ugel[[f'exd_{x}',f'{x}_c']].min(axis=1)  
    df_ugel[f'exd_{x}_n'] = df_ugel[f'exd_{x}'] - df_ugel[f'exd_{x}_c']

# Brecha sin racionalizacion

En esta brecha se financia todos los requerimientos

In [6]:
costo_oficinista = 12*1200+600+12*(1200*0.09)
costo_secretario = 12*1400+600+12*(1400*0.09)
costo_otros      = 12*1150+600+12*(1150*0.09)

brecha_sr = df_ugel.copy()
for x in personal:
    brecha_sr[f'brecha_sr_{x}'] = brecha_sr[f'req_{x}']
    if x == 'oficinista':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_oficinista
    elif x == 'secretario':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_secretario
    else:
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_otros


In [7]:
region_sr = brecha_sr.groupby('d_dpto').sum()
region_sr.style.format({'costo_brecha_sr_pers_limp_mant'  : 'S/{:20,.0f}', 
                        'costo_brecha_sr_pers_vigilancia' : 'S/{:20,.0f}', 
                        'costo_brecha_sr_coord_adm_ie'    : 'S/{:20,.0f}',
                        'costo_brecha_sr_oficinista'      : 'S/{:20,.0f}', 
                        'costo_brecha_sr_secretario'      : 'S/{:20,.0f}'
                       })

,pers_limp_mant_n,pers_limp_mant_c,pers_limp_mant,opt_pers_limp_mant,exd_pers_limp_mant,req_pers_limp_mant,pers_vigilancia_n,pers_vigilancia_c,pers_vigilancia,opt_pers_vigilancia,exd_pers_vigilancia,req_pers_vigilancia,coord_adm_ie_n,coord_adm_ie_c,coord_adm_ie,opt_coord_adm_ie,exd_coord_adm_ie,req_coord_adm_ie,oficinista_n,oficinista_c,oficinista,opt_oficinista,exd_oficinista,req_oficinista,secretario_n,secretario_c,secretario,opt_secretario,exd_secretario,req_secretario,exd_pers_limp_mant_c,exd_pers_limp_mant_n,exd_pers_vigilancia_c,exd_pers_vigilancia_n,exd_coord_adm_ie_c,exd_coord_adm_ie_n,exd_oficinista_c,exd_oficinista_n,exd_secretario_c,exd_secretario_n,brecha_sr_pers_limp_mant,costo_brecha_sr_pers_limp_mant,brecha_sr_pers_vigilancia,costo_brecha_sr_pers_vigilancia,brecha_sr_coord_adm_ie,costo_brecha_sr_coord_adm_ie,brecha_sr_oficinista,costo_brecha_sr_oficinista,brecha_sr_secretario,costo_brecha_sr_secretario
d_dpto,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AMAZONAS,296,30,326,322,75,71,0,43,43,263,0,220,26,4,30,183,7,160,19,0,19,47,11,39,20,2,22,5,19,2,26,49,0,0,2,5,0,11,2,17,71,"S/ 1,110,582",220,"S/ 3,441,240",160,"S/ 2,502,720",39,"S/ 635,544",2,"S/ 37,824"
ANCASH,969,143,1112,871,327,86,0,99,99,602,0,503,20,2,22,382,0,360,60,7,67,210,26,169,59,8,67,30,53,16,136,191,0,0,0,0,3,23,8,45,86,"S/ 1,345,212",503,"S/ 7,867,926",360,"S/ 5,631,120",169,"S/ 2,754,024",16,"S/ 302,592"
APURIMAC,331,28,359,316,94,51,0,46,46,201,0,155,0,0,0,152,0,152,64,7,71,72,36,37,24,3,27,10,21,4,23,71,0,0,0,0,7,29,3,18,51,"S/ 797,742",155,"S/ 2,424,510",152,"S/ 2,377,584",37,"S/ 602,952",4,"S/ 75,648"
AREQUIPA,709,141,850,783,194,127,4,75,79,535,1,457,23,3,26,353,19,346,144,12,156,175,64,83,87,11,98,17,83,2,131,63,1,0,3,16,12,52,11,72,127,"S/ 1,986,534",457,"S/ 7,148,394",346,"S/ 5,412,132",83,"S/ 1,352,568",2,"S/ 37,824"
AYACUCHO,396,66,462,464,68,70,0,58,58,312,0,254,20,4,24,201,4,181,35,6,41,113,18,90,31,6,37,14,28,5,42,26,0,0,4,0,2,16,6,22,70,"S/ 1,094,940",254,"S/ 3,973,068",181,"S/ 2,831,202",90,"S/ 1,466,640",5,"S/ 94,560"
CAJAMARCA,631,120,751,923,90,262,0,127,127,707,0,580,29,7,36,479,12,455,65,7,72,170,33,131,51,5,56,22,45,11,77,13,0,0,4,8,6,27,5,40,262,"S/ 4,098,204",580,"S/ 9,072,360",455,"S/ 7,117,110",131,"S/ 2,134,776",11,"S/ 208,032"
CALLAO,602,163,765,731,113,79,0,10,10,500,0,490,6,2,8,218,0,210,127,26,153,238,28,113,24,6,30,43,15,28,113,0,0,0,0,0,18,10,6,9,79,"S/ 1,235,718",490,"S/ 7,664,580",210,"S/ 3,284,820",113,"S/ 1,841,448",28,"S/ 529,536"
CUSCO,625,119,744,955,58,269,0,128,128,636,0,508,7,3,10,439,2,431,137,34,171,195,89,113,39,19,58,24,44,10,45,13,0,0,2,0,27,62,19,25,269,"S/ 4,207,698",508,"S/ 7,946,136",431,"S/ 6,741,702",113,"S/ 1,841,448",10,"S/ 189,120"
HUANCAVELICA,165,53,218,198,42,22,10,52,62,161,3,102,7,0,7,110,1,104,14,0,14,38,6,30,18,8,26,4,23,1,37,5,3,0,0,1,0,6,8,15,22,"S/ 344,124",102,"S/ 1,595,484",104,"S/ 1,626,768",30,"S/ 488,880",1,"S/ 18,912"


In [8]:
sr_costos = []
sr_personal = []

for x in personal:
    sr_personal.append(f'brecha_sr_{x}')
    sr_costos.append(f'costo_brecha_sr_{x}')

resumen_sr_personal = list(brecha_sr[sr_personal].sum())
resumen_sr_costos = list(brecha_sr[sr_costos].sum())
personal_resumen = ['Limpieza y mantenimiento','Vigilantes','Coordinador administrativo','Oficinista','Secretario']


dict_resumen = {'Perfiles': personal_resumen, 'Plazas': resumen_sr_personal, 'Costos': resumen_sr_costos} 
    
resumen_sr = pd.DataFrame(dict_resumen)
resumen_sr.loc['5'] = resumen_sr.sum()
resumen_sr.iloc[5, resumen_sr.columns.get_loc('Perfiles')] = 'Total'

print('En resumen: ')
resumen_sr.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })

En resumen: 


,Perfiles,Plazas,Costos
0,Limpieza y mantenimiento,"5,743","S/ 89,832,006"
1,Vigilantes,"14,281","S/ 223,383,402"
2,Coordinador administrativo,"8,355","S/ 130,688,910"
3,Oficinista,"4,236","S/ 69,029,856"
4,Secretario,477,"S/ 9,021,024"
5,Total,"33,092","S/ 521,955,198"


# Brecha con ordenamiento nacional

Criterios de ordenamiento:
1) Los **nombrados excedentes se reubican dentro de su propia UGEL a una IIEE con requerimiento:** El argumento de este movimiento es que al ser IIEE con mas de 140 alumnos son similares en sus caracteristicas

2) **Si aun existen nombrados excedentes ya no se reubican a una IIEE con requerimiento:** Esto se explica porque realizar un moviento dentro de la misma region puede ocasionar gastos de transporte asociados

3) **Las plazas contratadas o vacantes excedentes se mueven a nivel nacional una IIEE con requerimiento**

In [9]:
brecha_cr = df_ugel.copy()

for x in personal:
    # Brecha UGEL
    brecha_cr[f'brecha_ugel_{x}'] = brecha_cr[f'exd_{x}_n'] + brecha_cr[f'exd_{x}_c'] - brecha_cr[f'req_{x}']
    
    # Nombrados que no pueden ser reasignados
    brecha_cr[f'nom_exd_rig_{x}'] = brecha_cr[f'exd_{x}_n'] - brecha_cr[f'req_{x}'] 
    brecha_cr[f'nom_exd_rig_{x}'] = np.where(brecha_cr[f'nom_exd_rig_{x}']<0, 0 , brecha_cr[f'nom_exd_rig_{x}'] )

In [ ]:

# #Brecha regional
brecha_cr['req1'] = brecha_cr['Requerimientos'] - brecha_cr['Excedentes nombrados'] 
brecha_cr['req1'] = np.where(brecha_cr['req1']<0,0,brecha_cr['req1'])
brecha_cr['Brecha con Racio'] = brecha_cr['Excedentes contratados'] - brecha_cr['req1']


In [9]:
brecha_cr = df_ugel.copy()

# Brecha UGEL
brecha_cr['brecha_ugel'] = brecha_cr['exd_pers_limp_mant_n'] + brecha_cr['exd_pers_limp_mant_c'] - brecha_cr['req_pers_limp_mant']

# Nombrados que no pueden ser reasignados
brecha_cr['nom_exd_rig'] = brecha_cr['exd_pers_limp_mant_n'] - brecha_cr['req_pers_limp_mant'] 
brecha_cr['nom_exd_rig'] = np.where(brecha_cr['nom_exd_rig']<0, 0 , brecha_cr['nom_exd_rig'] )

brecha_cr.rename(columns={"pers_limp_mant_n": "Nombrados", 
                   "pers_limp_mant_c": "Contratados",
                    'pers_limp_mant' : 'Personal actual',
                   "opt_pers_limp_mant": "Personal optimo",
                   "exd_pers_limp_mant_n": "Excedentes nombrados",
                   "exd_pers_limp_mant_c": "Excedentes contratados",
                   'exd_pers_limp_mant' : 'Excedentes',
                   "req_pers_limp_mant": "Requerimientos",
                   'nom_exd_rig': 'Nombrados excedentes sin reubicar',
                   "brecha_ugel": "Brecha UGEL con racio",
                  }, inplace=True)

# #Brecha regional
brecha_cr['req1'] = brecha_cr['Requerimientos'] - brecha_cr['Excedentes nombrados'] 
brecha_cr['req1'] = np.where(brecha_cr['req1']<0,0,brecha_cr['req1'])
brecha_cr['Brecha con Racio'] = brecha_cr['Excedentes contratados'] - brecha_cr['req1']


In [10]:
# #Brecha regional
region_cr = brecha_cr.groupby('d_dpto').sum()
region_cr.drop(columns = ['req1','Brecha UGEL con racio'], inplace =  True)
region_cr

,Nombrados,Contratados,Personal actual,Personal optimo,Excedentes,Requerimientos,Excedentes contratados,Excedentes nombrados,Nombrados excedentes sin reubicar,Brecha con Racio
d_dpto,,,,,,,,,,
AMAZONAS,296.0,30.0,326.0,322.0,75.0,71.0,26.0,49.0,36.0,-32.0
ANCASH,969.0,143.0,1112.0,871.0,327.0,86.0,136.0,191.0,126.0,115.0
APURIMAC,331.0,28.0,359.0,316.0,94.0,51.0,23.0,71.0,63.0,-20.0
AREQUIPA,709.0,141.0,850.0,783.0,194.0,127.0,131.0,63.0,20.0,47.0
AYACUCHO,396.0,66.0,462.0,464.0,68.0,70.0,42.0,26.0,24.0,-26.0
CAJAMARCA,631.0,120.0,751.0,923.0,90.0,262.0,77.0,13.0,9.0,-181.0
CALLAO,602.0,163.0,765.0,731.0,113.0,79.0,113.0,0.0,0.0,34.0
CUSCO,625.0,119.0,744.0,955.0,58.0,269.0,45.0,13.0,3.0,-214.0
HUANCAVELICA,165.0,53.0,218.0,198.0,42.0,22.0,37.0,5.0,3.0,17.0


In [11]:
b_cr = np.absolute(region_cr['Brecha con Racio'].sum())
costo_cr = b_cr*(12*1150+600+12*(1150*0.09))

print(f'Financiar esta brecha de {b_cr:,} plazas costaria S/{costo_cr:,}')

Financiar esta brecha de 2,976.0 plazas costaria S/46,550,592.0


In [12]:
brecha_cr.drop(columns = ['req1', 'Brecha con Racio'], inplace =  True)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultados\Personal de limpieza.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
brecha_sr.to_excel(writer, sheet_name='UGEL sin racio')
region_sr.to_excel(writer, sheet_name='Region sin racio')

brecha_cr.to_excel(writer, sheet_name='UGEL con racio')
region_cr.to_excel(writer, sheet_name='Region con racio')

# Close the Pandas Excel writer and output the Excel file.
writer.save()